In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

import pandas as pd
import numpy as np

In [2]:
vowels = 'а у о ы и э я ю ё е'.split(' ')

In [3]:
df = pd.read_csv('all_accents.tsv', sep='\t', header=None)
df = df[~df[0].apply(lambda x: '-' in x and not x.startswith('-'))]
df = df[df[0].apply(lambda x: len(x) <= 20)]
df = df[~df[1].apply(lambda x: '^' not in x and len(set(x) & set(vowels)) > 0)]

chars = (set(list(''.join(df[0].values))))
corpus = {value: i for i, value in enumerate(sorted(chars))}
corpus_inv = {value: key for key, value in corpus.items()}

print(f'Всего слов: {df.shape[0]}')

Всего слов: 1669471


In [4]:
X = [[corpus[x] for x in word] for word in df[0]]
X = pad_sequences(X, 20, padding='post')
y = df[1].apply(lambda x: x.index('^') if '^' in x else -1)

In [5]:
model = Sequential([
    Input(shape=[20]),
    Embedding(len(corpus), 8),
    LSTM(128, return_sequences=True),
    LSTM(128),
    Dense(1),
])

model.compile('adam', 'mean_squared_error')

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 8)             280       
_________________________________________________________________
lstm (LSTM)                  (None, 20, 128)           70144     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 202,137
Trainable params: 202,137
Non-trainable params: 0
_________________________________________________________________


In [7]:
history = model.fit(X, y, epochs=100)

Epoch 1/100
52171/52171 [==============================] - 333s 6ms/step - loss: 1.3372
Epoch 2/100
52171/52171 [==============================] - 334s 6ms/step - loss: 0.7046
Epoch 3/100
52171/52171 [==============================] - 332s 6ms/step - loss: 0.5584
Epoch 4/100
52171/52171 [==============================] - 334s 6ms/step - loss: 0.4825
Epoch 5/100
52171/52171 [==============================] - 330s 6ms/step - loss: 0.4403
Epoch 6/100
52171/52171 [==============================] - 333s 6ms/step - loss: 0.4137
Epoch 7/100
52171/52171 [==============================] - 332s 6ms/step - loss: 0.3948
Epoch 8/100
52171/52171 [==============================] - 329s 6ms/step - loss: 0.3789
Epoch 9/100
52171/52171 [==============================] - 329s 6ms/step - loss: 0.3653
Epoch 10/100
52171/52171 [==============================] - 329s 6ms/step - loss: 0.3571
Epoch 11/100
52171/52171 [==============================] - 329s 6ms/step - loss: 0.3489
Epoch 12/100
52171/52171 [====

52171/52171 [==============================] - 332s 6ms/step - loss: 0.2044
Epoch 93/100
52171/52171 [==============================] - 328s 6ms/step - loss: 0.2040
Epoch 94/100
52171/52171 [==============================] - 328s 6ms/step - loss: 0.2028
Epoch 95/100
52171/52171 [==============================] - 328s 6ms/step - loss: 0.2034
Epoch 96/100
52171/52171 [==============================] - 328s 6ms/step - loss: 0.2007
Epoch 97/100
52171/52171 [==============================] - 328s 6ms/step - loss: 0.2024
Epoch 98/100
52171/52171 [==============================] - 328s 6ms/step - loss: 0.2016
Epoch 99/100
52171/52171 [==============================] - 328s 6ms/step - loss: 0.1999
Epoch 100/100
52171/52171 [==============================] - 332s 6ms/step - loss: 0.1987


In [8]:
test_words = ['привет', 'задание', 'телефон', 'признание', 'комната', 'уважение', 'табель', 'задница', 'морковь', 'желудь', 'язык', 'машиностроение', 'единообразие', 'леха', 'образование', 'миграция', 'желтый', 'деревья']
X_test = [[corpus[x] for x in word] for word in test_words]
X_test = pad_sequences(X_test, 20, padding='post')

y_pred = model.predict(X_test)
for i, [idx] in enumerate(y_pred):
    word = test_words[i]
    for j in np.argsort(np.abs(np.arange(len(word)) - idx)):
        if word[j] in vowels: break
    print(word[:j+1] + u'\u0301' + word[j+1:])

приве́т
зада́ние
телефо́н
призна́ние
ко́мната
уваже́ние
та́бель
за́дница
морко́вь
желу́дь
язы́к
машинострое́ние
единообра́зие
леха́
образова́ние
мигра́ция
же́лтый
дере́вья


In [9]:
test_words = ['приивет', 'сдание', 'обожуние', 'призенание', 'комнатка', 'уважка', 'табелек', 'задницкой', 'кекушка', 'облолился', 'затролировал']
X_test = [[corpus[x] for x in word] for word in test_words]
X_test = pad_sequences(X_test, 20, padding='post')

y_pred = model.predict(X_test)
for i, [idx] in enumerate(y_pred):
    word = test_words[i]
    for j in np.argsort(np.abs(np.arange(len(word)) - idx)):
        if word[j] in vowels: break
    print(word[:j+1] + u'\u0301' + word[j+1:])

прииве́т
сда́ние
обожу́ние
призена́ние
ко́мнатка
ува́жка
табе́лек
за́дницкой
ке́кушка
облоли́лся
затроли́ровал


In [10]:
from tensorflow.keras.models import load_model

model = load_model('striker.tf')

In [21]:
def print_striked(word: str):
    word = word.lower()
    if 'ё' in word:
        idx = word.index('ё')
    else:
        X = [[corpus[x] for x in word]]
        idx = model.predict(X)
    print(idx)

print_striked('келка')

[[-0.96517575]]


In [20]:
model.predict(X)

KeyboardInterrupt: 